In [3]:
import requests
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
load_dotenv()

# CONFIGS
API_KEY = os.getenv("CONGRESS_API_KEY")
BASE_URL = "https://api.congress.gov/v3"
START_DATE = "2020-01-01"
END_DATE = date.today().strftime("%Y-%m-%d")

keywords = ["energy", "grid", "resources", "electric", "power", 
            "utility", "renewable", "transmission", "coal", 
            "solar", "wind", "battery"]

# checks if the word contains any of the keywords
def contains_keyword(text: str) -> bool:
  # returns True if any keyword is in the text
  if not isinstance(text, str):
    return False
  text_lower = text.lower()
  return any(k in text_lower for k in keywords)

# fetches bill summaries from Congress API
def get_amendments(limit):
  url = f"{BASE_URL}/amendment"
  params = {
    "api_key": API_KEY,
    "fromDateTime": f"{START_DATE}T00:00:00Z",
    "toDateTime": f"{END_DATE}T23:59:59Z",
    "sort": "updateDate+desc",
    "limit": limit,
    "format": "json"
  }
  resp = requests.get(url, params=params)
  resp.raise_for_status()
  return resp.json().get("amendments", [])

def filter_amendments():
  amendments = get_amendments(limit=250)
  matched = []
  for a in amendments:
    # obtains information from the amendments
    latest_action = a.get("latestAction", {})
    text = latest_action.get("text", "")
    # checks keywords in both title and summary text
    if contains_keyword(text):
      matched.append({
        "action_date": latest_action.get("actionDate"),
        "text": text,
        "congress": a.get("congress"),
        "number": a.get("number"),
        "type": a.get("type"),
        "purpose": a.get("purpose"),
        "url": a.get("url"),
        "update_date": a.get("updateDate")
      })


  # converts list to dataframe
  amendments_df = pd.DataFrame(matched)

  # ensure all columns exist
  for col in ["action_date","action_desc","congress","number","type","purpose","url","update_date"]:
    if col not in amendments_df.columns:
      amendments_df[col] = pd.NA

  # stores times/dates as datetime
  amendments_df["action_date"] = pd.to_datetime(amendments_df["action_date"], errors="coerce")
  amendments_df["update_date"] = pd.to_datetime(amendments_df["update_date"], errors="coerce")

  return amendments_df.sort_values(by="action_date")

amendments_df = filter_amendments()
print(f"Total amendments matched: {len(amendments_df)}")

### RESULT
# no amendments related to energy recently...

Total amendments matched: 0
